In [1]:
import pandas as pd
import requests

In [2]:
df = pd.DataFrame(columns=['year', 'voted_count', 'registered_count', 'voted_pct'])

### Add in manually calculated 2014-15 results 

In [3]:
df.loc[len(df.index)] = ['2014', 2980, 18049, 100 * (2980/18049)]
df.loc[len(df.index)] = ['2015', 1578, 16492, 100 * (1578/16492)]

### Scrape 2016-21 results from county system

In [4]:
ELECTIONS = {
    '2016': 'https://results.enr.clarityelections.com/PA/Allegheny/63905/193472/json/status.json',
    '2017': 'https://results.enr.clarityelections.com/PA/Allegheny/71801/193474/json/status.json',
    '2018': 'https://results.enr.clarityelections.com/PA/Allegheny/92253/222754/json/status.json',
    '2019': 'https://results.enr.clarityelections.com//PA/Allegheny/98378/258755/json/status.json',
    '2020': 'https://results.enr.clarityelections.com//PA/Allegheny/106267/272321/json/status.json',
    '2021': 'https://results.enr.clarityelections.com/PA/Allegheny/111176/285575/json/status.json'
}

In [5]:
for election_year in ELECTIONS:
    r = requests.get(ELECTIONS[election_year], headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'
      }).json()
    
    precinct_names = [ precinct for precinct in r['P'] if 'Pittsburgh Ward 4' in precinct or 'PITTSBURGH WARD 4' in precinct ]
    precinct_indices = [ r['P'].index(precinct_name) for precinct_name in precinct_names ]

    voted = 0
    registered = 0

    for precinct_index in precinct_indices:
        voted += r['B'][precinct_index]
        registered += r['R'][precinct_index]

    df.loc[len(df.index)] = [election_year, voted, registered, 100 * (voted/registered)]

### Output results

In [6]:
df

,year,voted_count,registered_count,voted_pct
0,2014,2980,18049,16.510610
1,2015,1578,16492,9.568276
2,2016,11340,21142,53.637310
3,2017,1838,20743,8.860821
4,2018,7575,21266,35.620239
5,2019,2661,22088,12.047265
6,2020,8274,16795,49.264662
7,2021,2063,16191,12.741647


In [7]:
df[['year', 'voted_pct']].to_csv('pgh-ward-4.csv', index=False)